# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS project2
    WITH REPLICATION =
    {'class': 'SimpleStrategy','replication_factor':1}""")

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('project2')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### The first query: the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [9]:
#create the table artist_song_session
query1="CREATE TABLE IF NOT EXISTS artist_song_session"
query1=query1+"(sessionId int, itemInSession int, artist varchar, length float, song varchar, PRIMARY KEY(sessionId,itemInSession))"
try:
    session.execute(query1)
except Exception as e:
    print(e)
                    

### For this query, sessionId and itemInSession are considered as the composite key. Because they perfectly filters the data as per the query. 

In [10]:
# set up the CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query1 = "INSERT INTO artist_song_session(sessionId,itemInSession,artist,length,song)"
        query1 = query1 + "VALUES(%s,%s,%s,%s,%s)"
        session.execute(query1, (int(line[8]),int(line[3]),line[0],float(line[5]),line[9]))

### The second query: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [11]:
## Add in the SELECT statement to verify the data was entered into the table
query1="SELECT artist, song, length FROM artist_song_session WHERE sessionId=338 and itemInSession=4"
try:
    rows=session.execute(query1)
except Exception as e:
    print(e)
for row in rows:
    print (row.artist,row.song,row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [23]:
#create the table song_playlist_session
query2="CREATE TABLE IF NOT EXISTS song_playlist_session"
query2=query2+"(userId int, sessionId int, artist varchar, song varchar, itemInSession int, firstName varchar, lastName varchar, PRIMARY KEY(userId,sessionId,itemInSession))"
try:
    session.execute(query2)
except Exception as e:
    print(e)
                    

### For this query, userId, sessionId and itemInSession are considered as the composite key. Because they perfectly filters the data as per the query and song needs to be sorted by itemInSession.

In [24]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query2 = "INSERT INTO song_playlist_session(userId,sessionId,artist,song,itemInSession,firstName,lastName)"
        query2 = query2 + "VALUES(%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query2, (int(line[10]),int(line[8]),line[0], line[9], int(line[3]),line[1],line[4]))

In [25]:
## Add in the SELECT statement to verify the data was entered into the table
query2="SELECT artist, song, firstName, lastName FROM song_playlist_session WHERE userid=10 and sessionId=182"
try:
    rows=session.execute(query2)
except Exception as e:
    print(e)
for row in rows:
    print (row.artist,row.song,row.firstname,row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### The third query: every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [26]:
#create the table user_song_session
query3="CREATE TABLE IF NOT EXISTS user_song_session"
query3=query3+"(song varchar, userId int, firstName varchar, lastName varchar, PRIMARY KEY(song, userId))"
try:
    session.execute(query3)
except Exception as e:
    print(e)

### For this query, song and userId are considered as the composite key. Because they perfectly filters the data as per the query. 

In [27]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query3 = "INSERT INTO user_song_session(song, userId, firstName, lastName)"
        query3 = query3 + "VALUES(%s,%s,%s,%s)"
        session.execute(query3, (line[9],int(line[10]),line[1],line[4]))

In [28]:
## Add in the SELECT statement to verify the data was entered into the table
query3="SELECT firstName, lastName FROM user_song_session WHERE song='All Hands Against His Own'"
try:
    rows=session.execute(query3)
except Exception as e:
    print(e)
for row in rows:
    print (row.firstname,row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [18]:
query1="DROP table artist_song_session"
try:
    rows=session.execute(query1)
except Exception as e:
    print(e)

In [19]:
query2="DROP table song_playlist_session"
try:
    rows=session.execute(query2)
except Exception as e:
    print(e)

In [20]:
query3="DROP table user_song_session"
try:
    rows=session.execute(query3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()